# Unmixing signals with ICA

Unmixing sound signals is an example of cocktail party problem you are going to use for getting hands-on experience with ICA. You have 5 mixed sound sources in **mixed** folder (go check them out). Your goal is to unmix them.

In [46]:
import scipy.io.wavfile
import numpy as np

### Loading data from WAV files

Loading data from WAV files

In [2]:
dataset = []
for i in range(1,6):
    sample_rate, wav_data = scipy.io.wavfile.read('mixed/mix'+str(i)+'.wav')
    dataset.append(wav_data)

dataset = np.array(dataset).T
print(dataset.shape)
print(dataset[:10,:])

(53442, 5)
[[ 343 -546 -327 -275  612]
 [ 627 -840 -579 -124  890]
 [ 589 -725 -491 -115  989]
 [ 712 -887 -571  -24 1111]
 [ 589 -725 -491 -115  989]
 [ 268 -462 -146 -236  678]
 [ 107 -330   27 -296  522]
 [-214  -67  372 -416  211]
 [-214  -67  372 -416  211]
 [ 159 -206  -26 -233  445]]


Normalizing data

In [3]:
maxs = np.max(np.abs(dataset), axis=0).astype(np.int64)
data_normalized = 0.99 * dataset / maxs;
print(data_normalized[:10,:])

[[ 0.01046796 -0.01666328 -0.00997965 -0.00839268  0.01867752]
 [ 0.0191353  -0.02563581 -0.0176704  -0.00378433  0.02716175]
 [ 0.01797558 -0.02212614 -0.01498474 -0.00350966  0.03018311]
 [ 0.0217294  -0.02707019 -0.01742625 -0.00073245  0.03390641]
 [ 0.01797558 -0.02212614 -0.01498474 -0.00350966  0.03018311]
 [ 0.00817904 -0.01409969 -0.00445575 -0.00720244  0.02069176]
 [ 0.00326551 -0.01007121  0.00082401 -0.00903357  0.01593082]
 [-0.00653103 -0.00204476  0.011353   -0.01269583  0.00643947]
 [-0.00653103 -0.00204476  0.011353   -0.01269583  0.00643947]
 [ 0.00485249 -0.00628688 -0.00079349 -0.00711089  0.01358087]]


In [4]:
print(data_normalized.shape)

(53442, 5)


### Implementing ICA

Initializing unmixing matrix $ W $.

In [57]:
W = np.identity(5)

Implement learning unmixing matrix $ W $ with ICA.

In [41]:
W

array([[144.05023609,  33.41566068,  29.51285492,  28.34824301,
        -35.01738934],
       [-15.54245658,  98.80108428,  30.51544743,  24.90965236,
         17.74014687],
       [ 50.44656336,  28.25339094, -27.09973648, -63.54872037,
         88.4464861 ],
       [ -0.42359464,  67.94151121, -52.49415506, -40.12728613,
        -47.56832515],
       [-97.27683303, -22.79527218, -99.42609132,  59.49578269,
         42.1156579 ]])

In [51]:
np.linalg.norm(W)

298.3328863259075

In [60]:
# =============== TODO: Your code here ===============
# Implement learning unmixing matrix W with ICA. Do not forget to account for the dimensionality.
eps = 0.001
alpha = 0.0001
diff = 1

while diff > eps:
    additional = np.linalg.inv(W.T)
    
    grad = additional*data_normalized.shape[0]
    
    grad = grad + (1-2*(1/(1+np.exp(-W.dot(np.transpose(data_normalized)))))).dot(data_normalized)
    
    W_new = W + alpha*grad
    
    diff = np.linalg.norm(W_new-W)
    
    W = W_new
    
print("final precision is {}".format(diff))
    

# ====================================================

final precision is 9.988551088202237e-05


### Unmixing sounds

Use learned matrix $ W $ to unmix the sounds into separate data sources. Make sure you represent the resulting unmixing matrix in a way so that each row is a separate track (i.e. the matrix should have 5 rows).

In [71]:
# =============== TODO: Your code here ===============
# Use learned matrix W to unmix the sounds into separate data sources.

unmixed = W.dot(data_normalized.T)
# ====================================================

Saving unmixed sounds. Please note that some players may not support the resulting WAV format. If that is the case, you can use Winamp to play the unmixed sounds.

In [66]:
maxs = np.max(np.abs(unmixed), axis=1).reshape((5,1))
unmixed_normalized = 0.99 * unmixed / maxs;

for i in range(unmixed_normalized.shape[0]):
    track = unmixed_normalized[i,:]
    scipy.io.wavfile.write('unmixed/unmixed'+str(i)+'.wav', sample_rate, track)